In [2]:
import pandas as pd
df_train=pd.read_csv("/home/users/kostadin.mishev/phd/dataset/train/train.tsv",sep='\t',names=['id','label','a','text'])
df_test=pd.read_csv("/home/users/kostadin.mishev/phd/dataset/dev/dev.tsv",sep='\t',names=['id','label','a','text'])

In [3]:
y_train = df_train['label'].values
y_test = df_test['label'].values

In [4]:
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []

def stem(sentences):
    corpus = []
    for i,s in enumerate(sentences):
        review = re.sub('[^a-zA-Z]', ' ', sentences[i])
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
        review = ' '.join(review)
        corpus.append(review)
    return corpus;

X_train = stem(df_train['text'].values)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/users/kostadin.mishev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
X_test = stem(df_test['text'].values)

In [12]:
import pysentiment as ps
hiv4 = ps.HIV4()
tokens = hiv4.tokenize(X_train[1])
score = hiv4.get_score(tokens)
print(score)

{'Positive': 0, 'Negative': 0, 'Polarity': 0.0, 'Subjectivity': 0.0}


In [47]:
def get_scores(model,sentences):
    scores = list()
    for s in sentences:
        tokens = model.tokenize(s)
        score = model.get_score(tokens)
        pol = 0
        if (score['Polarity']>0):
            pol=1
        elif (score['Polarity']<0):
            pol=-1
        scores.append(pol)
    return scores;

hiv4 = ps.HIV4()
lm = ps.LM()

hiv4 = get_scores(lm,X_test)

In [49]:
for i,s in enumerate(hiv4):
    if (s==0):
        hiv4[i]=1-y_train[i]
    elif (s==-1):
        hiv4[i]=0
        

In [50]:
y_pred = hiv4

In [51]:
from sklearn.metrics import precision_score, matthews_corrcoef, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
print("accuracy:",accuracy_score(y_test, y_pred))
print("precision:",precision_score(y_test, y_pred))
print("recall:",recall_score(y_test, y_pred))
print("f1-score:",f1_score(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel() 
print(tp)
print(tn)
print(fp)
print(fn)
print("mcc:",matthews_corrcoef(y_test, y_pred))

accuracy: 0.5730593607305936
precision: 0.5769230769230769
recall: 0.547945205479452
f1-score: 0.5620608899297423
120
131
88
99
mcc: 0.146303391191891


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [ ]:
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

#Train the XGboost Model for Classification
model1 = xgb.XGBClassifier(n_estimators=1000, max_depth=8, learning_rate=0.01)

xgb_model = model1.fit(X_train, Y_train_labels)

Y_pred = xgb_model.predict(X_dev)

tn, fp, fn, tp = confusion_matrix(Y_dev_labels,Y_pred).ravel() 
mcc = matthews_corrcoef(Y_dev_labels, Y_pred)

print(tp)
print(tn)
print(fp)
print(fn)
print(mcc)

In [8]:
def accuracy_summary(pipeline, X_train, y_train, X_test, y_test):
    sentiment_fit = pipeline.fit(X_train, y_train)
    y_pred = sentiment_fit.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy score: {0:.2f}%".format(accuracy*100))
    return accuracy

In [10]:
import numpy as np
cv = CountVectorizer()
rf = RandomForestClassifier(class_weight="balanced")
n_features = np.arange(10000,25001,5000)

def nfeature_accuracy_checker(vectorizer=cv, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=rf):
    result = []
    print(classifier)
    print("\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Test result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, X_train, y_train, X_test, y_test)
        result.append((n,nfeature_accuracy))
    return result

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [12]:
print("Result for trigram with stop words (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3))

Result for trigram with stop words (Tfidf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


Test result for 10000 features
accuracy score: 79.91%
Test result for 15000 features
accuracy score: 77.63%
Test result for 20000 features
accuracy score: 79.00%
Test result for 25000 features
accuracy score: 78.08%


In [46]:
import keras
y_train_ = keras.utils.to_categorical(y_train,2)

In [55]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
import xgboost
cv = CountVectorizer()
X_train = cv.fit_and_transform(X_train)
gsearch1 = GridSearchCV(estimator = xgboost.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train, y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

SyntaxError: invalid syntax (<ipython-input-55-659060f2196e>, line 8)

In [83]:
from sklearn.metrics import classification_report

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import xgboost as xgb
cv = CountVectorizer()
#cv = TfidfVectorizer(max_features=30000)
#xgb = xgb.XGBClassifier(learning_rate=0.05, n_estimators=1000, objective='binary:logistic', max_depth=4, nthread=4)
#xgb = xgb.XGBClassifier(learning_rate=0.01, n_estimators=5000, objective='binary:logistic',silent=False, nthread=8)
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
svc = SVC();
pipeline = Pipeline([
        ('vectorizer', cv),
        ('classifier', svc)
    ])
#sentiment_fit = pipeline.fit(X_train, y_train_,classifier__nb_epoch = 10, classifier__batch_size=batch_size)
sentiment_fit = pipeline.fit(X_train, y_train)
y_pred = sentiment_fit.predict(X_test)

#print(classification_report(y_test, y_pred, target_names=['negative','positive']))

In [91]:
cv = TfidfVectorizer()
for c in [0.0025,0.025,0.1,0.25,1,10,100,200,1000]:
    print(c)
    svc = SVC(kernel='rbf', C=c)
    model = Pipeline([
        ('vectorizer', cv),
        ('classifier', svc)
    ])
    model.fit(X_train, y_train)
    Y_pred = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test,Y_pred).ravel() 
    mcc = matthews_corrcoef(y_test, Y_pred)

    print(tp)
    print(tn)
    print(fp)
    print(fn)
    print(mcc)
    print()

0.0025
212
24
195
7
0.15134597302460623

0.025
212
24
195
7
0.15134597302460623

0.1
212
22
197
7
0.13773065286109412

0.25
206
64
155
13
0.3058941789179756

1
186
179
40
33
0.6670074823163689

10
189
182
37
30
0.694418748712932

100
188
181
38
31
0.6852816599140776

200
188
181
38
31
0.6852816599140776

1000
188
181
38
31
0.6852816599140776



In [77]:
import numpy as np
y_pred = np.argmax(y_pred,axis=1)

In [84]:
from sklearn.metrics import precision_score, matthews_corrcoef, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
print("accuracy:",accuracy_score(y_test, y_pred))
print("precision:",precision_score(y_test, y_pred))
print("recall:",recall_score(y_test, y_pred))
print("f1-score:",f1_score(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel() 
print(tp)
print(tn)

print(fp)
print(fn)
print("mcc:",matthews_corrcoef(y_test, y_pred))

accuracy: 0.7831050228310502
precision: 0.7870370370370371
recall: 0.776255707762557
f1-score: 0.7816091954022988
170
173
46
49
mcc: 0.5662631784962306


In [36]:
## K-fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = pipeline, X= X_train, y = y_train,
                             cv = 10)
print("Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)"  % (accuracies.mean(), accuracies.std() * 2))

KeyboardInterrupt: 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/users/kostadin

In [47]:
max_fatures = 30000
#tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer = CountVectorizer(max_features=30000,ngram_range=(1, 3))
tokenizer.fit(X_train)
X_train = tokenizer.transform(X_train)
X_test = tokenizer.transform(X_test)

In [48]:
X_train[2]

<1x28346 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [10]:
X_train = pad_sequences(X_train,maxlen=30)
X_test = pad_sequences(X_test,maxlen=30)

In [11]:
X_test.shape

(438, 30)

In [49]:
embed_dim = 150
lstm_out = 200

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2,dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28346, 150)        4500000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               280800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 4,781,202
Trainable params: 4,781,202
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
vector=3531
embed_size=3531

from keras.models import Sequential
#from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 

model = Sequential()
#model.add(Dense(4096, activation='relu', input_shape=(vector,)))
#model.add(Dense(1024, activation='relu'))
#model.add(Dense(512, activation='relu', input_shape=(vector,)))
model.add(Dense(64, activation='relu', input_shape=(vector,)))
model.add(Dense(2, activation='softmax'))
model.summary()
#model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) # compile model with training parameters
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
num_classes=2
batch_size = 500
epochs = 50

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 64)                226048    
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 130       
Total params: 226,178
Trainable params: 226,178
Non-trainable params: 0
_________________________________________________________________


In [67]:
y_train_ = pd.get_dummies(y_train).values

In [51]:
y_train_ = np.argmax(y_train,axis=1)

In [52]:
y_test = pd.get_dummies(y_test).values

Exception: Data must be 1-dimensional

In [50]:
batch_size = 32
model.fit(X_train, y_train_, nb_epoch = 10, batch_size=batch_size, verbose = 2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


AttributeError: 'str' object has no attribute 'ndim'

In [35]:
y_pred=model.predict(X_test)

In [36]:
import numpy as np
y_test_ = np.argmax(y_test,axis=1)
y_pred_ = np.argmax(y_pred,axis=1)

In [37]:
from sklearn.metrics import precision_score, matthews_corrcoef, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
print("accuracy:",accuracy_score(y_test_, y_pred_))
print("precision:",precision_score(y_test_, y_pred_))
print("recall:",recall_score(y_test_, y_pred_))
print("f1-score:",f1_score(y_test_, y_pred_))
tn, fp, fn, tp = confusion_matrix(y_test_, y_pred_).ravel() 
print(tn)
print(tp)
print(fp)
print(fn)
print("mcc:",matthews_corrcoef(y_test_, y_pred_))

accuracy: 0.8127853881278538
precision: 0.8246445497630331
recall: 0.7945205479452054
f1-score: 0.8093023255813954
182
174
37
45
mcc: 0.6259885807570283


In [22]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,  42,  12,  23, 258,
       164,   5, 325,  86], dtype=int32)

In [23]:
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 1868,  175, 1140,  401, 1457], dtype=int32)

In [1]:
df_test.head()

NameError: name 'df_test' is not defined

In [25]:
import xgboost as xgb

#Train the XGboost Model for Classification
model1 = xgb.XGBClassifier()

xgb_model = model1.fit(X_train, y_train_)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

Y_pred = xgb_model.predict(X_test)



In [28]:
y_pred_ = Y_pred

In [29]:
from sklearn.metrics import precision_score, matthews_corrcoef, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
print("accuracy:",accuracy_score(y_test_, y_pred_))
print("precision:",precision_score(y_test_, y_pred_))
print("recall:",recall_score(y_test_, y_pred_))
print("f1-score:",f1_score(y_test_, y_pred_))
tn, fp, fn, tp = confusion_matrix(y_test_, y_pred_).ravel() 
print(tn)
print(tp)
print(fp)
print(fn)
print("mcc:",matthews_corrcoef(y_test_, y_pred_))

accuracy: 0.5228310502283106
precision: 0.5352112676056338
recall: 0.3470319634703196
f1-score: 0.42105263157894735
153
76
66
143
mcc: 0.04877642568991054


In [30]:
df_train.head()

,id,label,a,text
0,810,1,a,Tesco share price jumps as Q3 sales top estimates
1,4669,0,a,Diluted earnings per share ( EPS ) fell to EUR...
2,801,0,a,Barclays reports 8% fall in annual profits
3,4754,0,a,"Operating profit totalled EUR 1.22 mn , down f..."
4,1046,0,a,Tesco UK personnel director quits supermarket
